In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import time
import datetime
import requests
from tiingo import TiingoClient
from yahoofinancials import YahooFinancials
import os
import pandas_datareader.data as web

from tools import *

%config Completer.use_jedi = False

### Download stock general information (meta data)

In [2]:
useful_info = ['symbol', 'sector', 'industry', 'fullTimeEmployees', 'country',
               'maxAge',  'trailingAnnualDividendYield', 
               'payoutRatio', 'navPrice', 'totalAssets', 
               'trailingAnnualDividendRate', 'dividendRate', 
               'beta',  'startDate', 'priceHint', 'marketCap', 'dividendYield', 'exchange', 'shortName', 'longName', 
                'market', 'beta3Year', '52WeekChange', 'morningStarRiskRating', 'forwardEps', 
               'revenueQuarterlyGrowth', 'sharesOutstanding', 'fundInceptionDate', 'annualReportExpenseRatio', 
               'bookValue', 'sharesShort', 'sharesPercentSharesOut', 'fundFamily', 'lastFiscalYearEnd', 
               'heldPercentInstitutions', 'netIncomeToCommon', 'trailingEps', 
               'lastDividendValue', 'SandP52WeekChange', 'priceToBook', 
               'heldPercentInsiders', 'nextFiscalYearEnd', 'mostRecentQuarter', 'shortRatio', 
               'sharesShortPreviousMonthDate', 'floatShares', 'enterpriseValue', 'threeYearAverageReturn', 
               'lastSplitDate', 'lastSplitFactor', 'legalType', 'lastDividendDate', 
               'morningStarOverallRating', 'earningsQuarterlyGrowth', 'dateShortInterest', 'pegRatio',
               'lastCapGain', 'shortPercentOfFloat', 'sharesShortPriorMonth', 
               'impliedSharesOutstanding', 'category', 'fiveYearAverageReturn', 'longBusinessSummary'
               ]

def get_meta_data_information(symbols, useful_info):
    useful_infos = []
    for i, symbol in enumerate(symbols):
        print(f"{i+1}:{symbol}", end="  ")
        try:
            ticker = yf.Ticker(symbol)
            info = ticker.info
            useful_info = {x:info[x] for x in useful_info}
            useful_infos.append(useful_info)
        except Exception as e:
            print(e)
    return useful_infos


# meta_data = get_meta_data_information(symbols, useful_info)
# meta_data = pd.DataFrame(meta_data)
# meta_data.to_csv("All_US_stocks_above2B_MCap_meta_data.csv", index=False)
# meta_data.head(3)

In [3]:
# pd.read_csv("../Data/+Stocks_lists/IVV_holdings.csv")
# len(symbols)

### Get list of symbols (tickers)

In [4]:
# s1 = pd.read_excel("../Data/+Stocks_lists/2021-12. CanadianStock_top100.xlsx")['Symbol']
# s2 = pd.read_csv("../Data/+Stocks_lists/companiesmarketcap.com - Largest Canadian companies by market capitalization.csv")['Symbol']
# s1 = [x[:-3].upper() for x in s1]
# np.array((set(s1) - set(s2)))

In [5]:
ticker_family = 'SP500' # 'Nasdaq_NYSE' 

if ticker_family == 'SP500':
    # sp500 = get_sp500_list()
    # symbols = sp500['Symbol'].values
    sp500 = pd.read_excel("../Data/+Stocks_lists/ssga_holdings-daily-us-en-spy.xlsx")
    symbols = sp500['Ticker'].values

elif ticker_family == 'US_above2B':
    symbols_df = pd.read_excel("../Data/+Stocks_lists/All_US_stocks_above2B_MCap.xlsx")
    symbols = symbols_df['Symbol'].values[:]

elif ticker_family == 'Nasdaq':
    stock_exchange = 'Nasdaq'
    nasdaq = pd.read_csv("Assorted/20210615. nasdaq_screener_1623818257700.csv")[['Symbol', 'Name', 'Market Cap', 'Last Sale']]
    bins = [-1, 3e+8, 2e+9, 10e+9, 100e+9, 10000e+9]
    nasdaq['Mkt_cap_label'] = pd.cut(nasdaq['Market Cap'], bins=bins, labels=['Penny','Small', 'Medium', 'Large', "Mega"])
    symbols = nasdaq[nasdaq['Mkt_cap_label'].isin(['Mega','Large', 'Medium'])]['Symbol'].values

elif ticker_family == 'TSX':
    stock_exchange = 'tsx'
    tsx_symbols_df = pd.read_excel("../Data/+Stocks_lists/CanadianStock_Dec2021_top100.xlsx")
    symbols = tsx_symbols_df['Symbol'].astype(str).values 
    # symbols = [x.replace('.','-') + '.to' for x in symbols ] # if '.' not in x
    # symbols.remove('nan.to')
    # symbols.append('NA.to')

# elif ticker_family == "SPX_Sectors":
#     symbols= ['XLB', 'XLV', 'XLP', 'XLY', 'XLE', 'XLF', 'XLI', 'XLK', 'XLU', 'XLRE', 'XLC']

    
elif ticker_family == 'Indices':
    symbols = ['SPY', '^DJI', '^GSPC', 'NDX',
              'XLB', 'XLV', 'XLP', 'XLY', 'XLE', 'XLF', 'XLI', 'XLK', 'XLU', 'XLRE', 'XLC']

print('Number of Symbols =', len(symbols))
# symbols_df[:4]

Number of Symbols = 503


In [6]:
def download_stock_price_data(symbol, start, end, option='yahoo_finance', interval='1d', verbose=False):
    try:
        if option == 'yahoo_finance':
            # https://github.com/ranaroussi/yfinance
            # 
            # print(f"Download {symbol} data with Yahoo Finance API")
            # Option 1
            # Interval: 1m, 5m, 15m, 30m, 60m, 1h, 1d, 1wk,1mo,
            # auto_adjust = True: so all the presented prices areadjusted for potential corporate actions, such as splits.
            data = yf.download(symbol, start=start, end=end, progress=False, interval=interval)

        elif option == "YahooFinancials":
            # Option 3
            yahoo_financials = YahooFinancials(symbol)
            data = yahoo_financials.get_historical_price_data(start_date=start, end_date=end, 
                                                              time_interval='daily')
            data = pd.DataFrame(data[symbol]['prices'])        


        elif option == "alpha_vintage":
            ALPHA_VANTAGE_API_KEY = 'I3Z5IQNTGCQ7IED4'  # J4RADOTW5DBN344U
            print(f"{i+1}/{len(symbols)}: Download {symbol} data with Alpha Vintage using Pandas data_reader API")
            data = web.DataReader(symbol, "av-daily-adjusted", # av-daily-adjusted / av-daily
                                  start=start, end=end,
                                  api_key=ALPHA_VANTAGE_API_KEY)

        return data
    
    except Exception as e:
        print(f"Got an error: {e}")


def download_stock_price_data_wrapper(symbols, 
                                      start_date, prev_end_date, new_end_date, fake_end_date,
                                      option='yahoo_finance', interval='1d',
                                      save_folder='../Data/Daily/US_Market/latest'):

    interval_dict = {'1d': 'daily', '1h':'hourly', '60min':'hourly', '60m':'hourly', '1mo':'Monthly'}
    incomplete_downloads = dict()
    for i, symbol in enumerate(symbols[:]):
        print(f"{i+1} : {symbol} \t", end='')

        time_period = f"{start_date.replace('-', '')}_to_{prev_end_date.replace('-', '')}"
        save_file_name_old = f'{save_folder}/{symbol}_USD_{time_period}_{interval_dict[interval]}_{option}.csv'

        time_period = f"{start_date.replace('-', '')}_to_{new_end_date.replace('-', '')}"
        save_file_name_new = f'{save_folder}/{symbol}_USD_{time_period}_{interval_dict[interval]}_{option}.csv'

        if os.path.exists(save_file_name_new):
            data = pd.read_csv(save_file_name_new).set_index("Date")
            last_row_date = data.iloc[-1, :].name
            if last_row_date == new_end_date:
                print('{:>2d}: {:<10s} Has already been downloaded until {:<12s}'.format(i+1, symbol, last_row_date))
                continue
            else:
                print('{:>2d}: {:<10s} Download from {:<12s} to {:^12s}'.format(i+1, symbol, last_row_date, new_end_date))
                new_data = download_stock_price_data(symbol, start=last_row_date, end=fake_end_date, interval=interval)
                new_data.index = new_data.index.astype(str)
                merged_data = pd.concat((data, new_data))
                merged_data = merged_data[~merged_data.index.duplicated(keep='first')]
                os.remove(save_file_name_new)
                merged_data.to_csv(save_file_name_new)
                if merged_data.index[-1] != new_end_date:
                    incomplete_downloads[symbol] = merged_data.index[-1]

        elif os.path.exists(save_file_name_old):
            data = pd.read_csv(save_file_name_old).set_index("Date")
            last_row_date = data.iloc[-1, :].name
            print('{:>2d}: {:<10s} Download from {:<12s} to {:^12s}'.format(i+1, symbol, last_row_date, new_end_date))
            new_data = download_stock_price_data(symbol, start=last_row_date, end=fake_end_date, interval=interval)
            new_data.index = new_data.index.astype(str)
            merged_data = pd.concat((data, new_data))
            merged_data = merged_data[~merged_data.index.duplicated(keep='first')]
            os.remove(save_file_name_old)
            merged_data.to_csv(save_file_name_new)
            if merged_data.index[-1] != new_end_date:
                incomplete_downloads[symbol] = merged_data.index[-1]

        else:
            print('{:>2d}: {:<10s} Download from scratch'.format(i+1, symbol))
            data = download_stock_price_data(symbol, start=start_date, end=fake_end_date, interval=interval)
            if len(data) > 0:
                data.to_csv(save_file_name_new)
            else:
                incomplete_downloads[symbol] = 0
                continue
            if data.index[-1] != new_end_date:
                incomplete_downloads[symbol] = data.index[-1]

    print("Done")

In [7]:
# from pandas_datareader.nasdaq_trader import get_nasdaq_symbols
# nasdaq_symbols_df = get_nasdaq_symbols()
# print(len(nasdaq_symbols_df))
# nasdaq_symbols_df.head(2)

## Downalod Data

In [10]:
%%time
today = datetime.datetime.today().strftime('%Y-%m-%d')

start_date = "2000-01-01"
prev_end_date = "2022-03-23"
new_end_date = str(today) # "2022-02-02" # str(today)
fake_end_date = str(datetime.date.today() + datetime.timedelta(days=1)) # "2022-02-03"

download_stock_price_data_wrapper(symbols[:], start_date, prev_end_date, new_end_date, fake_end_date, interval='1d',
                                 save_folder='../Data/SP500/')

1 : MSFT 	 1: MSFT       Download from scratch
2 : AAPL 	 2: AAPL       Download from scratch
3 : NVDA 	 3: NVDA       Download from scratch
4 : AMZN 	 4: AMZN       Download from scratch
5 : META 	 5: META       Download from scratch
6 : GOOGL 	 6: GOOGL      Download from scratch
7 : GOOG 	 7: GOOG       Download from scratch
8 : BRKB 	 8: BRKB       Download from scratch

1 Failed download:
- BRKB: No data found, symbol may be delisted
9 : LLY 	 9: LLY        Download from scratch
10 : AVGO 	10: AVGO       Download from scratch
11 : JPM 	11: JPM        Download from scratch
12 : XOM 	12: XOM        Download from scratch
13 : TSLA 	13: TSLA       Download from scratch
14 : UNH 	14: UNH        Download from scratch
15 : V 	15: V          Download from scratch
16 : PG 	16: PG         Download from scratch
17 : MA 	17: MA         Download from scratch
18 : JNJ 	18: JNJ        Download from scratch
19 : MRK 	19: MRK        Download from scratch
20 : HD 	20: HD         Download from scrat

171 : NEM 	171: NEM        Download from scratch
172 : MCHP 	172: MCHP       Download from scratch
173 : TRV 	173: TRV        Download from scratch
174 : CPRT 	174: CPRT       Download from scratch
175 : WMB 	175: WMB        Download from scratch
176 : OKE 	176: OKE        Download from scratch
177 : URI 	177: URI        Download from scratch
178 : SPG 	178: SPG        Download from scratch
179 : ADSK 	179: ADSK       Download from scratch
180 : KMB 	180: KMB        Download from scratch
181 : AEP 	181: AEP        Download from scratch
182 : SRE 	182: SRE        Download from scratch
183 : ALL 	183: ALL        Download from scratch
184 : OXY 	184: OXY        Download from scratch
185 : ROST 	185: ROST       Download from scratch
186 : O 	186: O          Download from scratch
187 : HES 	187: HES        Download from scratch
188 : AFL 	188: AFL        Download from scratch
189 : MET 	189: MET        Download from scratch
190 : JCI 	190: JCI        Download from scratch
191 : TEL 	191: TE

338 : STLD 	338: STLD       Download from scratch
339 : ULTA 	339: ULTA       Download from scratch
340 : PPL 	340: PPL        Download from scratch
341 : FE 	341: FE         Download from scratch
342 : STE 	342: STE        Download from scratch
343 : APTV 	343: APTV       Download from scratch
344 : INVH 	344: INVH       Download from scratch
345 : HBAN 	345: HBAN       Download from scratch
346 : BRO 	346: BRO        Download from scratch
347 : AEE 	347: AEE        Download from scratch
348 : CBOE 	348: CBOE       Download from scratch
349 : OMC 	349: OMC        Download from scratch
350 : TYL 	350: TYL        Download from scratch
351 : ILMN 	351: ILMN       Download from scratch
352 : MKC 	352: MKC        Download from scratch
353 : DRI 	353: DRI        Download from scratch
354 : FSLR 	354: FSLR       Download from scratch
355 : CINF 	355: CINF       Download from scratch
356 : CNP 	356: CNP        Download from scratch
357 : SYF 	357: SYF        Download from scratch
358 : WBD 	3

503 : NWS 	503: NWS        Download from scratch
Done
Wall time: 8min 12s


In [11]:
today = datetime.datetime.today().strftime('%Y-%m-%d')
start_date = "2000-01-01"
prev_end_date = "2022-03-23"
new_end_date = str(today) # "2022-02-02" # str(today)
fake_end_date = str(datetime.date.today() + datetime.timedelta(days=1)) # "2022-02-03"

download_stock_price_data_wrapper(['BRK-B'], start_date, prev_end_date, new_end_date, fake_end_date, interval='1d',
                                 save_folder='../Data/SP500/')

1 : BRK-B 	 1: BRK-B      Download from scratch
Done


In [ ]:
tsx_symbols_df = pd.read_excel("../Data/+Stocks_lists/CanadianStock_Dec2021_top100.xlsx")
selected_symbols = ['TIXT.to', 'CDAY.to', 'FSV.to', 'CAE.to', 'AEM.to', 'Shop.to']
selected_symbols = ['AEM.to', 'CCO.to', 'NCM.to', 'CDAY.to', 'CP.to', 'TIXT.to', 'BEP-UN.to']
selected_symbols = ['CSU.to', 'WCN.to', 'TRI.to','ATD.to', 'NTR.to']

tsx_symbols_df.set_index('Symbol').loc[selected_symbols][['Company', 'Sector', 'Market Cap', 'Last Sale']]

## Download SPX and analyze it

In [ ]:
download_stock_price_data_wrapper(['^GSPC'], 
                                  '1970-01-01', '2021-12-31', '2021-12-31', '2022-01-01', interval='1mo',
                                 save_folder='../Data/Daily/US_Market/Indices')

In [ ]:
spy_mo = download_stock_price_data('^GSPC', start='1970-01-01', end='2021-12-04', interval='1mo')
spy_mo

In [ ]:
n = 3
spy_pc = spy_mo.dropna()[['Close']].pct_change(n).dropna() * 100
spy_pc.columns = [f'Percent of change in {n} months']
spy_pc['MovingAvg_3'] = spy_pc[f'Percent of change in {n} months'].rolling(3).mean() # , min_periods=1
spy_pc['MovingAvg_6'] = spy_pc[f'Percent of change in {n} months'].rolling(6).mean() # , min_periods=1
spy_pc['MovingAvg_12'] = spy_pc[f'Percent of change in {n} months'].rolling(12).mean() # , min_periods=1
spy_pc['MovingAvg_18'] = spy_pc[f'Percent of change in {n} months'].rolling(18).mean() # , min_periods=1

spy_pc.tail(5)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

fig, ax = plt.subplots(figsize=(18, 6), dpi=100)
x = 0
spy_pc.iloc[:, x:x+1].plot(kind='line', ax=ax, x_compat=True) # mdates.MonthLocator()
ax.grid(which='major', axis='both')
# ax.xaxis.set_major_locator(plt.MaxNLocator( 30))
# ax.xaxis.set_major_locator(mdates.MonthLocator())
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=12))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
# fig.autofmt_xdate()
fig.savefig("SPY_chage_3_months.jpg")

In [ ]:
fig, ax = plt.subplots(figsize=(18, 6))
spy_pc[-100:].plot(kind='bar', ax=ax, lw=2, )
ax.grid(which='major')


In [ ]:
iwv_mo = download_stock_price_data('IWV', start='1970-01-01', end='2021-11-23', interval='1mo')

In [ ]:
iwv_mo_pc = iwv_mo.dropna()[['Close']].pct_change(12).dropna() * 100
iwv_mo_pc